In [1]:
from denovo_utils.parsers.converters import SpectralisParser, DenovoEngineConverter
from denovo_utils.utils.pandas import (
    get_decoy_status,
    collapse_casanovo_score,
    get_spectralis_score,
    get_psm_type,
    amino_acid_converter,
    drop_charge
)
from denovo_utils.parsers.constants import EXTENSIONS

import pandas as pd
import os

from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
from psm_utils import PSMList, PSM

from glob import glob

Modification already exists in ModificationsDB. Skipping.


In [2]:
df = pd.read_csv("/home/samva/Doctorate/data_directory/PXD028735/denovo_results/merged/LFQ_Orbitrap_DDA_Condition_A_Sample_Alpha_01.csv")

/tmp/ipykernel_1867/860113767.py:1: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/samva/Doctorate/data_directory/PXD028735/denovo_results/merged/LFQ_Orbitrap_DDA_Condition_A_Sample_Alpha_01.csv")


In [5]:
from denovo_utils.parsers.converters import DenovoEngineConverter

conv = DenovoEngineConverter.select("contranovo")
df = conv.parse(
    result_path="/home/samva/Doctorate/data_directory/tests/denovo_output/contranovo/test3.contranovo.mztab",
    mgf_path="/home/samva/Doctorate/data_directory/tests/mgf_reformatted/test3.mgf"
).to_dataframe()

100%|██████████| 36/36 [00:00<00:00, 3542.99it/s]


In [13]:
from denovo_utils.parsers.constants import ALL_MODIFICATION_LABELS, MODIFICATION_MAPPING

[label_e for label_e, label_map in ALL_MODIFICATION_LABELS.items() if label_e not in MODIFICATION_MAPPING["instanovo"].keys()]

['C(+57.02)', 'C', 'M(ox)', 'M(+15.99)', 'N(+.98)', 'Q(+.98)']

In [3]:
print(df.shape)
df.head(3)

(607933, 13)


,spectrum_id,proforma,sequence,run,source,score,qvalue,is_decoy,metadata,rescoring_features,precursor_mz,retention_time,protein_list
0,controllerType=0 controllerNumber=1 scan=85406,WAGNANELNAAYAADGYAR/2,WAGNANELNAAYAADGYAR,LFQ_Orbitrap_DDA_Condition_A_Sample_Alpha_01,sage,1.263158,0.000032,False,{},"{'expmass': 1996.9054, 'calcmass': 1996.9027, ...",999.460525,93.72429,"['sp|P06169|PDC1_YEAST', 'sp|P16467|PDC5_YEAST..."
1,controllerType=0 controllerNumber=1 scan=86089,ANC[UNLMOD:4]LDSTVDAETVFAR/2,ANCLDSTVDAETVFAR,LFQ_Orbitrap_DDA_Condition_A_Sample_Alpha_01,sage,1.251166,0.000032,False,{},"{'expmass': 1767.8107, 'calcmass': 1767.8096, ...",884.913175,94.26285,"['sp|P15646|FBRL_YEAST', 'tr|A0A0L8VSN0|A0A0L8..."
2,controllerType=0 controllerNumber=1 scan=80518,NQGDEEGTELDTLQFR/2,NQGDEEGTELDTLQFR,LFQ_Orbitrap_DDA_Condition_A_Sample_Alpha_01,sage,1.248334,0.000032,False,{},"{'expmass': 1850.8317, 'calcmass': 1850.8284, ...",926.423675,89.92643,['sp|O15160|RPAC1_HUMAN']


In [ ]:
root_data = "/home/samva/Doctorate/data_directory/PXD028735/denovo_results/merged"
root_mgf = "/home/samva/Doctorate/data_directory/PXD028735/mgf/Orbitrap_QE/reformatted"

filenames = sorted([
    x.split("/")[-1].split(".")[0] for x in glob(os.path.join(root_data,"*.csv"))
])

for filename in filenames:

    df = pd.read_csv(os.path.join(root_data, f"{filename}.csv"))
    mgf_path = os.path.join(root_mgf, f"{filename}.mgf")

    fig, ax = plt.subplots(1, 1, figsize=(15,4))
    ax.set_title(filename)
    sns.barplot(
        df.groupby("source").match_type.value_counts(normalize=True).reset_index(),
        x="source",
        hue="match_type",
        y="proportion",
        hue_order=["Match", "Isobaric", "Better", "Worse"],
        ax=ax
    )

    # Annotate the bars
    for p in ax.patches:
        height = p.get_height()  # Get the height of each bar
        ax.annotate(f'{height:.0%}',  # Format as percentage with two decimal places
                    (p.get_x() + p.get_width() / 2., height),  # Position the text in the center of the bar
                    ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),  # Slightly offset the text above the bar
                    textcoords='offset points')

    